In [187]:
import langchain
import openai
import streamlit
import json
import os
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2
load_dotenv() # loading all the environment variables from the .env file

True

In [188]:
from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
KEY=os.getenv("API_KEY")

sk-proj-Vzlgl5qjl7fgYlMLcweHT3BlbkFJbV22XBGW9reIwradH5JB


In [189]:
# Initializing the language model
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.3)

In [190]:
# Example of a quiz
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },

}

In [191]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [192]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [193]:
# chaining the input prompt and the llm
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [194]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity.
If the quiz os not at per with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students.
Quiz_MCQs:
{quiz}

Check from an expert English grammar of the above quiz:
"""

In [195]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template = TEMPLATE
)

In [196]:
# chaining the input prompt and the llm
review_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="review", verbose=True)

In [197]:
# Creating an object of sequential chain
# Used to connect both the prompts together
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text","number","subject","tone","response_json"], output_variables=["subject", "quiz"], verbose=True,)

In [198]:
file_path=r"C:\Users\sahil\Downloads\NLP Couses\NLP Basic Course\Data Science Natural Language Processing NLP in Python\GenAI Projects\MCQ generator\data.txt"

In [199]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [200]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]

From a theoretical 

In [201]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [202]:
NUMBER=5
SUBJECT="data science"
TONE="simple"

In [203]:
#  How to startup tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone":TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of

In [204]:
# Tracking the API expenses using the inbuilt API functions itself
print(f"Total tokens: {cb.total_tokens}")
print(f"Prompt tokens: {cb.prompt_tokens}")
print(f"Completion tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total tokens: 2875
Prompt tokens: 2196
Completion tokens: 679
Total Cost: 0.0021165000000000003


In [205]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFr

In [206]:
quiz = response.get("quiz")
quiz = json.loads(quiz)

In [209]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices": options, "Correct": correct})

In [210]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: The study of artificial intelligence concerned with the development and study of statistical algorithms | b: The study of computer hardware | c: The study of human cognitive processes | d: The study of quantum mechanics',
  'Correct': 'a'},
 {'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Donald Hebb | b: Tom M. Mitchell | c: Arthur Samuel | d: Walter Pitts',
  'Correct': 'c'},
 {'MCQ': 'What is the mathematical foundation of machine learning?',
  'Choices': 'a: Data mining | b: Artificial neural networks | c: Mathematical optimization | d: Natural language processing',
  'Correct': 'c'},
 {'MCQ': 'What is the purpose of modern-day machine learning?',
  'Choices': 'a: To classify data based on models and make predictions for future outcomes | b: To study human cognitive processes | c: To develop computer hardware | d: To analyze historical data',
  'Correct': 'a'},
 {'MCQ': 'What did Raytheon Company develop in 

In [215]:
# converting into a dataframe
quiz = pd.DataFrame(quiz_table_data)

In [216]:
# converting into csv file which will be created in the same directory
quiz.to_csv("machinelearning.csv", index=False)

In [218]:
quiz

,MCQ,Choices,Correct
0,What is machine learning?,a: The study of artificial intelligence concer...,a
1,Who coined the term 'machine learning'?,a: Donald Hebb | b: Tom M. Mitchell | c: Arthu...,c
2,What is the mathematical foundation of machine...,a: Data mining | b: Artificial neural networks...,c
3,What is the purpose of modern-day machine lear...,a: To classify data based on models and make p...,a
4,What did Raytheon Company develop in the early...,a: A learning machine with punched tape memory...,a


In [220]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

ValueError: Invalid format string